In [1]:
# Dependencies and Setup
import pandas as pd
import os
from config2 import cleaned_csv, destination_zip, col_types
import datetime as dt

# Raw dataset is too large to host on github (100MB limit), 
# ∴ download csv from link below and place zipped file in Resources folder to recreate the following steps.
# https://www.kaggle.com/sobhanmoosavi/us-accidents

# path to zip
zip_path = os.path.join("Resources","199387_1319582_bundle_archive.zip")



# Extract CSVs into DataFrames



# 1. --- US_Accidents CSV-----

In [2]:
# read the csv into pandas via the zip path above. automatically selects the csv residing within the zip
accidents_df = pd.read_csv(zip_path, 
                           usecols=col_types.keys(), 
                           dtype=col_types, 
                           parse_dates=['Start_Time'],
                           infer_datetime_format=True)

accidents_df.head()

,Start_Time,End_Time,State,Country,Temperature(F)
0,2016-02-08 05:46:00,2016-02-08 11:00:00,OH,US,36.9
1,2016-02-08 06:07:59,2016-02-08 06:37:59,OH,US,37.9
2,2016-02-08 06:49:27,2016-02-08 07:19:27,OH,US,36.0
3,2016-02-08 07:23:34,2016-02-08 07:53:34,OH,US,35.1
4,2016-02-08 07:39:07,2016-02-08 08:09:07,OH,US,36.0


# 2. Transform the Accident_CSV Into DataFrame

In [5]:
# To reduce th esize of the file we filter down to apllicable= rows that 
# Dataset starts on Feb 8 2016, to have data in whole month increments, we start in March 2016.
# Dataset goes to June 30 2020, to have equal representation of months, we end < March 2020
raw_len = len(accidents_df)

start_date = pd.to_datetime('2016-03-01 00:00:00')
cutoff_date = pd.to_datetime('2020-03-01 00:00:00')

# drop rows that are before the start date or after the cutoff date
accidents_df = accidents_df.drop(accidents_df[(accidents_df.Start_Time < start_date) | (accidents_df.Start_Time >= cutoff_date)].index)

print(f"""Starting length: {raw_len}
Ending length: {len(accidents_df)}
Trimmed {raw_len - len(accidents_df)}
Start date: {accidents_df.Start_Time.dt.date.min()}, End date: {accidents_df.Start_Time.dt.date.max()}""")

Starting length: 3151163
Ending length: 3151163
Trimmed 0
Start date: 2016-03-01, End date: 2020-02-29


In [6]:
# Create New [Date] & [Sunrise/Sunset] Columns

new_dates, new_times = zip(*[(d.date(), d.time()) for d in accidents_df['Start_Time']])
accidents_df = accidents_df.assign(Date=new_dates, Day_Night =new_times)
accidents_df.head()

,Start_Time,End_Time,State,Country,Temperature(F),Date,Day_Night
425,2016-03-01 05:05:42,2016-03-01 05:35:42,OH,US,34.0,2016-03-01,05:05:42
426,2016-03-01 06:14:44,2016-03-01 06:44:44,OH,US,37.9,2016-03-01,06:14:44
427,2016-03-01 07:30:18,2016-03-01 08:00:18,OH,US,37.0,2016-03-01,07:30:18
428,2016-03-01 07:31:53,2016-03-01 08:01:53,OH,US,35.6,2016-03-01,07:31:53
429,2016-03-01 07:46:03,2016-03-01 08:16:03,OH,US,37.0,2016-03-01,07:46:03


In [11]:
# Rename Temperature(F) column to just Temp

accidents_df = accidents_df.rename(columns={"Temperature(F)":"Temperature"})
accidents_df

,State,Country,Temperature,Date,Day_Night
425,OH,US,34.0,2016-03-01,05:05:42
426,OH,US,37.9,2016-03-01,06:14:44
427,OH,US,37.0,2016-03-01,07:30:18
428,OH,US,35.6,2016-03-01,07:31:53
429,OH,US,37.0,2016-03-01,07:46:03
...,...,...,...,...,...
3513612,CA,US,86.0,2019-08-23,18:03:25
3513613,CA,US,70.0,2019-08-23,19:11:30
3513614,CA,US,73.0,2019-08-23,19:00:21
3513615,CA,US,71.0,2019-08-23,19:00:21


In [7]:
#Drop 'Start_Time' &'End_Time' Columns 

accidents_df.drop(accidents_df.iloc[:, 0:2], inplace = True, axis = 1)
accidents_df.reset_index()
accidents_df.head()

,State,Country,Temperature(F),Date,Day_Night
425,OH,US,34.0,2016-03-01,05:05:42
426,OH,US,37.9,2016-03-01,06:14:44
427,OH,US,37.0,2016-03-01,07:30:18
428,OH,US,35.6,2016-03-01,07:31:53
429,OH,US,37.0,2016-03-01,07:46:03


In [8]:
# write Transformed Df to csv in zip
compression = dict(method='zip',
                   archive_name=cleaned_csv)  
accidents_df.to_csv(destination_zip, compression=compression)
